<a href="https://colab.research.google.com/github/Tizona1099/ADA-DHOxSS/blob/master/Dash_app_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 40.2 MB/s eta 0:00:00


In [ ]:
pip install dash-bootstrap-components

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 3.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from dash import Dash, html, dash_table, dcc, callback, Output, Input
from dash_bootstrap_components.themes import BOOTSTRAP
import os
import geopandas as gpd

In [ ]:
allData = pd.read_excel('SectorSurveysData_updated.xlsx')


In [ ]:
gdf_sp = gpd.read_file('RGN_DEC_2023_EN_BFC.shp')
gdf = gpd.read_file('RGN_DEC_2023_EN_BFC.shx')
gdf_dbf = gpd.read_file('RGN_DEC_2023_EN_BFC.dbf')

In [ ]:
list(allData.columns)

['UserID',
 'Archon_code',
 'Repository Name',
 'Local authority name (Governing body name):',
 'Is your archive service part of a larger organisation?',
 'Postcode',
 'Please select the local authorities for which you provide an archive service:',
 'Gwynedd',
 'Castell-nedd Port Talbot',
 'Abertawe',
 'Blaenau Gwent',
 'Bridgend',
 'Caerphilly',
 'Cardiff',
 'Carmarthenshire',
 'Conwy',
 'Merthyr Tydfil',
 'Monmouthshire',
 'Newport',
 'Pembrokeshire',
 'Rhondda Cynon Taf',
 'Vale of Glamorgan',
 'Torfaen',
 'Wrexham',
 'How are your services provided?',
 'Is your archive service co-located with another service?',
 'Total number of staff',
 'Total number of qualified archivists',
 'Total number of non-qualified archivists',
 'Total number of qualified curators',
 'Total number of records management staff',
 'Total number of conservation staff',
 'Total number of education and learning officers',
 'Total number of support staff',
 'Total number of project staff',
 'Total number of onsi

In [ ]:
'''
Staff overview with table and additional graphs
'''

# binned data
bins = [0, 1, 5, 10, 15, 20, 25, 30]
labels = ['0-1', '1.1-5', '5.1-10', '10.1-15', '15.1-20', '20.1-25', '25.1-30']
allData['archivists grouped'] = pd.cut(allData['Total number of qualified archivists'], bins=bins, labels=labels, include_lowest=True)
archivistBins = allData['archivists grouped'].value_counts(sort=False).reset_index().rename(columns={"index": "archivists", 0: "count"})

# Function to calculate percentages by category
def staff_percentages_by_category(df, cat_col, column_name):
    results = []

    # Filter out NaN values
    df_filtered = df[df[column_name].notna()]

    # Group by category
    grouped_data = df_filtered.groupby(cat_col)

    # Iterate over each group
    for category, group in grouped_data:
        total_archives = len(group)

        # Calculate counts for zero and non-zero qualified archivists
        zero_qualified_count = (group[column_name] == 0).sum()
        non_zero_qualified_count = (group[column_name] > 0).sum()

        # Calculate percentages
        zero_qualified_perc = round((zero_qualified_count / total_archives) * 100, 2)
        non_zero_qualified_perc = round((non_zero_qualified_count / total_archives) * 100, 2)

        job_name = column_name.replace('Total number of ', '').strip()

        # Append results to the list
        results.append({
            f'{cat_col}': category,
            job_name: 'None',
            'Count': zero_qualified_count,
            'Percentage': zero_qualified_perc
        })
        results.append({
            f'{cat_col}': category,
            job_name: 'Yes',
            'Count': non_zero_qualified_count,
            'Percentage': non_zero_qualified_perc
        })

    # Create a DataFrame from the results
    result_df = pd.DataFrame(results)
    return result_df

# Function to calculate % archives having zero staff vs some staff
def calc_zero_perc(df, column_name):
    # Filter out NaN values
    filtered_data = df[df[column_name].notna()]

    # Calculate total number of archives after filtering out NaN values
    total_archives = len(filtered_data)

    # Calculate counts for zero and non-zero values
    zero_count = (filtered_data[column_name] == 0).sum()
    non_zero_count = (filtered_data[column_name] > 0).sum()

    # Calculate percentages
    zero_percentage = round((zero_count / total_archives) * 100, 2)
    non_zero_percentage = round((non_zero_count / total_archives) * 100, 2)

    job_name = column_name.replace('Total number of ', '').strip()

    # Create a new DataFrame
    data = {
        job_name: [f'No {job_name}', f'Has {job_name}'],
        'Count': [zero_count, non_zero_count],
        'Percentage': [zero_percentage, non_zero_percentage]
    }
    result_df = pd.DataFrame(data)
    result_df = result_df.sort_values(by='Percentage', ascending=False)
    return result_df

def volunteers_bar_chart(df, group_by_column, target_cols):
    # Select only columns for summing, and include the grouping column

    grouped_df = df.groupby(group_by_column)[target_cols].sum().reset_index()

    # Melt the DataFrame to long format
    melted_df = pd.melt(grouped_df, id_vars=[group_by_column], value_vars=['Total number of onsite volunteers', 'Total number of online volunteers'],
                        var_name='Volunteer Type', value_name='Count')

    # Sort by the count of onsite volunteers
    melted_df['sort_order'] = melted_df.apply(lambda x: x['Count'] if x['Volunteer Type'] == 'Total number of onsite volunteers' else 0, axis=1)
    melted_df = melted_df.sort_values(by=['sort_order', group_by_column], ascending=[False, True])

    return melted_df


df = allData

agg_cols = ['Total number of staff',
 'Total number of qualified archivists',
 'Total number of non-qualified archivists',
 'Total number of qualified curators',
 'Total number of records management staff',
 'Total number of conservation staff',
 'Total number of education and learning officers',
 'Total number of project staff',
 'Total number of onsite volunteers',
 'Total number of online volunteers',
 'Total number of volunteer hours',]

stats = df[agg_cols].agg(['count', 'max', 'min', 'sum','mean', 'median']).round(2)

stats = stats.transpose()
new_col_names = ['responses', 'highest', 'lowest', 'sum', 'average', 'median']
stats.columns = new_col_names
stats=stats.reset_index(names='workforce')

df_table = stats

vol_cols = ['Total number of onsite volunteers', 'Total number of online volunteers']


# Initialize the Dash app
app = Dash(__name__, external_stylesheets=[BOOTSTRAP])

# App layout
app.layout = html.Div([
    html.Div(children='Staff Overview', style={'textAlign': 'center', 'color': 'blue', 'fontSize': 30, 'margin': '20px 0'}),

    html.Div(children=[
        dash_table.DataTable(data=df_table.to_dict('records'), page_size=5, style_table={'overflowX': 'auto', 'width': '70%', 'margin': '0 auto'},
        style_cell={'textAlign': 'center'}
        )
    ]),

    html.Div([
      html.Div([
            dcc.Graph(id='bin-graph', style={'height': '600px', 'margin': '10px auto'})
        ], style={'width': '80%'}
    )], style={'textAlign': 'centre'}),

    html.Div(children=[
        dcc.RadioItems(options=['Category', 'Postcode Region'],
                       value='Category', inline=True,
                       id='category-selector', style={'margin': '20px 0'})
    ], style={'textAlign': 'left'}),

    html.Div([
        dcc.Dropdown(
            id='job-selector',
            options=[{'label': col, 'value': col} for col in df.columns if col.startswith('Total number of ')],
            value=[col for col in df.columns if col.startswith('Total number of ')][1],
            style={'margin': '20px 0'}
        )
    ], style={'textAlign': 'left'}),

    html.Div([
        html.Div([
            dcc.Graph(id='category-graph', style={'height': '600px', 'margin': '30px auto'})
        ], style={'width': '33%', 'display': 'inline-block', 'verticalAlign': 'top'}),

        html.Div([
            dcc.Graph(id='overall-graph', style={'height': '600px', 'margin': '30px auto'})
        ], style={'width': '65%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ], style={'textAlign': 'right'}),

        html.Div([
        dcc.Graph(id='volunteer-graph', style={'height': '500px'})
    ], style={'textAlign': 'center', 'padding': 10, 'flex':1})
])

# Callback to update the graphs
@callback(
    [Output('category-graph', 'figure'),
     Output('overall-graph', 'figure'),
     Output('bin-graph', 'figure'),
     Output('volunteer-graph', 'figure')],
    [Input('category-selector', 'value'),
     Input('job-selector', 'value')]
)
def update_graphs(selected_category, selected_job):
    # Calculate percentages by category using the selected job
    qualified_percentages_by_category = staff_percentages_by_category(df, selected_category, selected_job)
    qualified_percentages_by_category = qualified_percentages_by_category.sort_values(by='Percentage', ascending=False)

    # Plot the category graph
    category_fig = px.bar(qualified_percentages_by_category,
                          y=selected_category,
                          x='Percentage',
                          color=selected_job.replace('Total number of ', '').strip(),
                          color_discrete_map={
                              'Yes': '#4C78A8',
                              'None': 'goldenrod'},
                          text='Percentage',
                          barmode='stack',
                          orientation='h',
                          height=600,
                          labels={selected_category: 'Category', 'Count': 'Count'},
                          title=f'{selected_job.replace("Total number of ", "").strip().title()} per Category')

    category_fig.update_traces(texttemplate='%{text:.2f}%', textposition='inside')
    category_fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide',
                               xaxis_title='Percentage', yaxis_title='Category',
                               template='plotly_white')

    # Calculate overall percentages
    overall_percentages = calc_zero_perc(df, selected_job)

    # Plot the overall graph
    overall_fig = px.bar(overall_percentages,
                        x=overall_percentages.columns[0],
                        y='Count',
                        text='Percentage',
                        height=600,
                        color=overall_percentages.columns[0],
                        color_discrete_map={
                             f'Has {selected_job.replace("Total number of ", "").strip()}': '#4C78A8',
                             f'No {selected_job.replace("Total number of ", "").strip()}': 'goldenrod'},
                         title=f'Percentage of Archives with {selected_job.replace("Total number of ", "").strip().title()}')

    overall_fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
    overall_fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide',
                              xaxis_title= f'{selected_job.replace("Total number of ", "").strip()}', yaxis_title='Count',
                              template='plotly_white')

    # Plot binned data graph
    bin_fig = px.bar(archivistBins, x='archivists grouped', y='count',
             labels={'archivists grouped': 'archivists grouped', 'count': 'count'},
             title='Number of Qualified Archivists Grouped')
    bin_fig.update_traces(marker_color='#4C78A8')
    bin_fig.update_layout(xaxis_title='Number of Qualified Archivists (FTE)', yaxis_title='Count',
                  template='plotly_white')

    # Plot volunteer graph
    vol_df= volunteers_bar_chart(df, selected_category, vol_cols)

    vol_fig = px.bar(vol_df,
                     y=selected_category,
                     x='Count',
                     color='Volunteer Type',
                     barmode='stack',
                     orientation='h',
                     height=600,
                     color_discrete_map={
                              'Total number of onsite volunteers': '#4C78A8',
                              'Total number of online volunteers': 'rgb(136, 204, 238)'},
                     title='Onsite and Online Volunteers by {}'.format(selected_category))
    vol_fig.update_layout(yaxis={'categoryorder':'total ascending'}, margin=dict(l=30, r=30, t=100, b=20), template='plotly_white')


    return overall_fig, category_fig, bin_fig, vol_fig

# Run the app
if __name__ == '__main__':
    app.run(jupyter_mode="external")


Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
def calc_zero_perc(df, column_name):
    # Filter out NaN values
    filtered_data = df[df[column_name].notna()]

    # Calculate total number of archives after filtering out NaN values
    total_archives = len(filtered_data)

    # Calculate counts for zero and non-zero values
    zero_count = (filtered_data[column_name] == 0).sum()
    non_zero_count = (filtered_data[column_name] > 0).sum()

    # Calculate percentages
    zero_percentage = round((zero_count / total_archives) * 100, 2)
    non_zero_percentage = round((non_zero_count / total_archives) * 100, 2)

    ncolumn_name = column_name.replace(" in the last financial year", "").strip().lower()
    print(ncolumn_name)

    # Create a new DataFrame
    data = {
        ncolumn_name: [f'Zero {ncolumn_name}', f'Some {ncolumn_name}'],
        'Count': [zero_count, non_zero_count],
        'Percentage': [zero_percentage, non_zero_percentage]
    }
    result_df = pd.DataFrame(data)
    result_df = result_df.sort_values(by='Percentage', ascending=False)
    return result_df

x = calc_zero_perc(allData, 'Total revenue expenditure in the last financial year')
x


total revenue expenditure


,total revenue expenditure,Count,Percentage
1,Some total revenue expenditure,127,71.75
0,Zero total revenue expenditure,50,28.25


In [ ]:
'''
Finance overview with map

'''

mask = (allData['TNA Region'] == 'Yorkshire')
allData.loc[mask,'Postcode Region'] = allData.loc[mask,'TNA Region']
allData.loc[allData["Postcode Region"] == "Yorkshire", "Postcode Region"] = "Yorkshire and The Humber"
allData.loc[allData["Postcode Region"] == "Greater London", "Postcode Region"] = "London"

df = allData

agg_cols = [ 'Total revenue expenditure in the last financial year',
 'Total capital expenditure in the last financial year',
 'Total income in the last financial year',]

stats = df[agg_cols].agg(['count', 'max', 'min', 'sum','mean', 'median']).round(2)

stats = stats.transpose()
new_col_names = ['responses', 'highest', 'lowest', 'sum', 'average', 'median']
stats.columns = new_col_names
stats=stats.reset_index(names='finance')

df_table = stats


bins = [0, 1000, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 500000, 1000000, 2000000]
labels = ['0-1000', '1001-10000', '10001-20000', '20001-30000', '30001-40000', '40001-50000', '50001-60000', '60001-70000', '70001-80000', '80001-90000', '90001-100000', '100001-500000', '500001-100000', '100001-200000']
df['income grouped'] = pd.cut(df['Total income in the last financial year'], bins=bins, labels=labels, include_lowest=True)
financeBins = df['income grouped'].value_counts(sort=False).reset_index().rename(columns={"index": "income", 0: "count"})

def calc_zero_perc(df, column_name):
    # Filter out NaN values
    filtered_data = df[df[column_name].notna()]

    # Calculate total number of archives after filtering out NaN values
    total_archives = len(filtered_data)

    # Calculate counts for zero and non-zero values
    zero_count = (filtered_data[column_name] == 0).sum()
    non_zero_count = (filtered_data[column_name] > 0).sum()

    # Calculate percentages
    zero_percentage = round((zero_count / total_archives) * 100, 2)
    non_zero_percentage = round((non_zero_count / total_archives) * 100, 2)

    ncolumn_name = column_name.replace(" in the last financial year", "").strip().lower()

    # Create a new DataFrame
    data = {
        ncolumn_name: [f'Zero {ncolumn_name}', f'Some {ncolumn_name}'],
        'Count': [zero_count, non_zero_count],
        'Percentage': [zero_percentage, non_zero_percentage]
    }
    result_df = pd.DataFrame(data)
    result_df = result_df.sort_values(by='Percentage', ascending=False)
    return result_df

# Initialize the Dash app
app = Dash(__name__, external_stylesheets=[BOOTSTRAP])

# App layout
app.layout = html.Div([
    html.Div(children='Finance Overview', style={'textAlign': 'center', 'color': 'green', 'fontSize': 30, 'margin': '20px 0'}),

    html.Div(children=[
        dash_table.DataTable(data=df_table.to_dict('records'), page_size=5, style_table={'overflowX': 'auto', 'width': '70%', 'margin': '0 auto'},
        style_cell={'textAlign': 'center'}
        )
    ]),

    html.Div([
      html.Div([
            dcc.Graph(id='fin-graph', style={'height': '600px', 'margin': '10px auto'})
        ], style={'width': '80%'}
    )], style={'textAlign': 'centre'}),

    html.Div([
        dcc.Dropdown(
            id='f-selector',
            options=[{'label': col, 'value': col} for col in df.columns if col.endswith('financial year')],
            value=[col for col in df.columns if col.endswith('financial year')][2],
            style={'margin': '20px 0'}
        )
    ], style={'textAlign': 'left'}),

      html.Div([
        html.Div([
        dcc.Graph(id='map-graph', style={'height': '600px'})
    ], style={'textAlign': 'center', 'padding': 10, 'flex':1}),

        html.Div([
        dcc.Graph(id='fin-category-graph', style={'height': '600px', 'margin': '10px auto'})
        ], style={'width': '35%', 'display': 'inline-block', 'verticalAlign': 'top'}),

        html.Div([
        dcc.Graph(id='zero-fin-graph', style={'height': '600px', 'margin': '10px auto'})
        ], style={'width': '60%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ], style={'textAlign': 'right'})


])

# Callback to update the graphs
@callback(
    [Output('map-graph', 'figure'),
     Output('fin-graph', 'figure'),
     Output('fin-category-graph', 'figure'),
     Output('zero-fin-graph', 'figure')],
    Input('f-selector', 'value')
)

def update_graphs(selected_job):
  df1 = df.groupby('Postcode Region')[selected_job].sum().round(2).reset_index()
  merged_gdf = gdf_sp.merge(df1, left_on='RGN23NM', right_on='Postcode Region')
  merged_gdf = merged_gdf.to_crs("WGS84")
  merged_gdf.set_index('RGN23NM', inplace=True)

  map_fig = px.choropleth(
    merged_gdf,
    geojson=merged_gdf.geometry,
    locations=merged_gdf.index,
    #featureidkey="properties.RGN23CD",  # Adjust this to match the GeoJSON properties key
    color=selected_job,
    color_continuous_scale="gnbu",
    #range_color=(merged_gdf['Total income in the last financial year'].min(), merged_gdf['Total income in the last financial year'].max()),
    labels={selected_job: selected_job, 'RGN23NM': 'Region'},
    title=f'{selected_job} by Region'
)

  # Update the layout to display the map
  map_fig.update_geos(fitbounds='geojson', visible=False)
  map_fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})

  fin_fig = px.bar(financeBins, x='income grouped', y='count',
             labels={'income grouped': '	Income grouped', 'count': 'Count'},
             title='Total Income Grouped')
  fin_fig.update_layout(xaxis_title='	Income grouped', yaxis_title='Count',
                  template='plotly_white',
                  width=800, height=600)
  fin_fig.update_traces(marker_color='rgb(102,166,30)')


    # Calculate overall percentages
  zero_percentages = calc_zero_perc(df, selected_job)
  ncolumn_name = selected_job.replace(" in the last financial year", "").strip().lower()

    # Plot the overall graph
  zero_fig = px.bar(zero_percentages,
                        x=zero_percentages.columns[0],
                        y='Count',
                        text='Percentage',
                        height=600,
                        #color=zero_percentages.columns[0],
                        color_discrete_map={
                             f'Some {ncolumn_name}': 'rrgb(115, 175, 72)',
                             f'Zero {ncolumn_name}': 'rgb(225, 124, 5)'},
                         title=f'Percentage of Archives with<br>{selected_job.title()}')

  zero_fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
  zero_fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide',
                              xaxis_title='', yaxis_title='Count',
                              template='plotly_white')
  #zero_fig.update_xaxes(showticklabels=False)

  df1 = df.groupby('Category')[selected_job].sum().round(2).reset_index()
  df1 = df1.sort_values(by=selected_job, ascending=True)
  fcat_fig = px.bar(df1, x=selected_job, y='Category',text=selected_job,
             labels={selected_job: f'{selected_job.replace("in the last financial year", "").strip()}'},
             title=f'{selected_job.replace("in the last financial year", "").strip().title()} Per Category')
  fcat_fig.update_traces(texttemplate='%{text:.3s}', textposition='none', marker_color='rgb(166,216,84)')
  fcat_fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide',
                  #width=1100,
                  xaxis_title=f'{selected_job.replace("in the last financial year", "").strip()}', yaxis_title='Category',
                  template='plotly_white')



  # Show the map
  return map_fig, fin_fig, zero_fig, fcat_fig

  # Run the app
if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>

In [41]:
'''
more tabs
'''
bins = [0, 1, 5, 10, 15, 20, 25, 30]
labels = ['0-1', '1.1-5', '5.1-10', '10.1-15', '15.1-20', '20.1-25', '25.1-30']
allData['archivists grouped'] = pd.cut(allData['Total number of qualified archivists'], bins=bins, labels=labels, include_lowest=True)
archivistBins = allData['archivists grouped'].value_counts(sort=False).reset_index().rename(columns={"index": "archivists", 0: "count"})

agg_cols = ['Total number of staff',
 'Total number of qualified archivists',
 'Total number of non-qualified archivists',
 'Total number of qualified curators',
 'Total number of records management staff',
 'Total number of conservation staff',
 'Total number of education and learning officers',
 'Total number of project staff',
 'Total number of onsite volunteers',
 'Total number of online volunteers',
 'Total number of volunteer hours',]

stats = df[agg_cols].agg(['count', 'max', 'min', 'sum','mean', 'median']).round(2)

stats = stats.transpose()
new_col_names = ['responses', 'highest', 'lowest', 'sum', 'average', 'median']
stats.columns = new_col_names
stats=stats.reset_index(names='workforce')

df_table = stats

vol_cols = ['Total number of onsite volunteers', 'Total number of online volunteers']

#functions:
# Function to calculate percentages by category
def staff_percentages_by_category(df, cat_col, column_name):
    results = []

    # Filter out NaN values
    df_filtered = df[df[column_name].notna()]

    # Group by category
    grouped_data = df_filtered.groupby(cat_col)

    # Iterate over each group
    for category, group in grouped_data:
        total_archives = len(group)

        # Calculate counts for zero and non-zero qualified archivists
        zero_qualified_count = (group[column_name] == 0).sum()
        non_zero_qualified_count = (group[column_name] > 0).sum()

        # Calculate percentages
        zero_qualified_perc = round((zero_qualified_count / total_archives) * 100, 2)
        non_zero_qualified_perc = round((non_zero_qualified_count / total_archives) * 100, 2)

        job_name = column_name.replace('Total number of ', '').strip()

        # Append results to the list
        results.append({
            f'{cat_col}': category,
            job_name: 'None',
            'Count': zero_qualified_count,
            'Percentage': zero_qualified_perc
        })
        results.append({
            f'{cat_col}': category,
            job_name: 'Yes',
            'Count': non_zero_qualified_count,
            'Percentage': non_zero_qualified_perc
        })

    # Create a DataFrame from the results
    result_df = pd.DataFrame(results)
    return result_df

# Function to calculate % archives having zero staff vs some staff
def calc_zero_perc(df, column_name):
    # Filter out NaN values
    filtered_data = df[df[column_name].notna()]

    # Calculate total number of archives after filtering out NaN values
    total_archives = len(filtered_data)

    # Calculate counts for zero and non-zero values
    zero_count = (filtered_data[column_name] == 0).sum()
    non_zero_count = (filtered_data[column_name] > 0).sum()

    # Calculate percentages
    zero_percentage = round((zero_count / total_archives) * 100, 2)
    non_zero_percentage = round((non_zero_count / total_archives) * 100, 2)

    job_name = column_name.replace('Total number of ', '').strip()

    # Create a new DataFrame
    data = {
        job_name: [f'No {job_name}', f'Has {job_name}'],
        'Count': [zero_count, non_zero_count],
        'Percentage': [zero_percentage, non_zero_percentage]
    }
    result_df = pd.DataFrame(data)
    result_df = result_df.sort_values(by='Percentage', ascending=False)
    return result_df

def volunteers_bar_chart(df, group_by_column, target_cols):
    # Select only columns for summing, and include the grouping column

    grouped_df = df.groupby(group_by_column)[target_cols].sum().reset_index()

    # Melt the DataFrame to long format
    melted_df = pd.melt(grouped_df, id_vars=[group_by_column], value_vars=['Total number of onsite volunteers', 'Total number of online volunteers'],
                        var_name='Volunteer Type', value_name='Count')

    # Sort by the count of onsite volunteers
    melted_df['sort_order'] = melted_df.apply(lambda x: x['Count'] if x['Volunteer Type'] == 'Total number of onsite volunteers' else 0, axis=1)
    melted_df = melted_df.sort_values(by=['sort_order', group_by_column], ascending=[False, True])

    return melted_df

def calc_fzero_perc(df, column_name):
    # Filter out NaN values
    filtered_data = df[df[column_name].notna()]

    # Calculate total number of archives after filtering out NaN values
    total_archives = len(filtered_data)

    # Calculate counts for zero and non-zero values
    zero_count = (filtered_data[column_name] == 0).sum()
    non_zero_count = (filtered_data[column_name] > 0).sum()

    # Calculate percentages
    zero_percentage = round((zero_count / total_archives) * 100, 2)
    non_zero_percentage = round((non_zero_count / total_archives) * 100, 2)

    column_name = column_name.lower()

    # Create a new DataFrame
    data = {
        column_name: [f'Zero {column_name}', f'Some {column_name}'],
        'Count': [zero_count, non_zero_count],
        'Percentage': [zero_percentage, non_zero_percentage]
    }
    result_df = pd.DataFrame(data)
    return result_df

df=allData




app = Dash(__name__)

tabs_styles = {
    'height': '44px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'fontWeight': 'bold'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#119DFF',
    'color': 'white',
    'padding': '6px'
}

app.layout = html.Div([
    dcc.Tabs(id="tabs-styled-with-inline", value='tab-1', children=[
        dcc.Tab(label='Tab 1', value='tab-1', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Tab 2', value='tab-2', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Tab 3', value='tab-3', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Tab 4', value='tab-4', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Tab 5', value='tab-5', style=tab_style, selected_style=tab_selected_style),
    ], style=tabs_styles),
    html.Div(id='tabs-content-inline')
])

@callback(Output('tabs-content-inline', 'children'),
            Input('tabs-styled-with-inline', 'value')
          )

def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            html.H3('Staffing'),
            html.Div(children='Staff Overview', style={'textAlign': 'center', 'color': 'blue', 'fontSize': 30, 'margin': '20px 0'}),

    html.Div(children=[
        dash_table.DataTable(data=df_table.to_dict('records'), page_size=5, style_table={'overflowX': 'auto', 'width': '70%', 'margin': '0 auto'},
        style_cell={'textAlign': 'center'}
        )
    ]),

    html.Div([
      html.Div([
            dcc.Graph(id='bin-graph', style={'height': '600px', 'margin': '10px auto'})
        ], style={'width': '80%'}
    )], style={'textAlign': 'centre'}),

    html.Div(children=[
        dcc.RadioItems(options=['Category', 'Postcode Region'],
                       value='Category', inline=True,
                       id='category-selector', style={'margin': '20px 0'})
    ], style={'textAlign': 'left'}),

    html.Div([
        dcc.Dropdown(
            id='job-selector',
            options=[{'label': col, 'value': col} for col in df.columns if col.startswith('Total number of ')],
            value=[col for col in df.columns if col.startswith('Total number of ')][1],
            style={'margin': '20px 0'}
        )
    ], style={'textAlign': 'left'}),

    html.Div([
        html.Div([
            dcc.Graph(id='category-graph', style={'height': '600px', 'margin': '30px auto'})
        ], style={'width': '33%', 'display': 'inline-block', 'verticalAlign': 'top'}),

        html.Div([
            dcc.Graph(id='overall-graph', style={'height': '600px', 'margin': '30px auto'})
        ], style={'width': '65%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ], style={'textAlign': 'right'}),

        html.Div([
        dcc.Graph(id='volunteer-graph', style={'height': '500px'})
    ], style={'textAlign': 'center', 'padding': 10, 'flex':1})
        ])
    elif tab == 'tab-2':
        return html.Div([
            html.H3('Tab content 2')
        ])
    elif tab == 'tab-3':
        return html.Div([
            html.H3('Tab content 3')
        ])
    elif tab == 'tab-4':
        return html.Div([
            html.H3('Tab content 4')
        ])
    elif tab == 'tab-5':
        return html.Div([
            html.H3('Finance')
        ])


# Callback to update the graphs
@callback(
    [Output('category-graph', 'figure'),
     Output('overall-graph', 'figure'),
     Output('bin-graph', 'figure'),
     Output('volunteer-graph', 'figure')],
    [Input('category-selector', 'value'),
     Input('job-selector', 'value')]
)

def update_graphs(selected_category, selected_job):
    # Calculate percentages by category using the selected job
    qualified_percentages_by_category = staff_percentages_by_category(df, selected_category, selected_job)
    qualified_percentages_by_category = qualified_percentages_by_category.sort_values(by='Percentage', ascending=False)

    # Plot the category graph
    category_fig = px.bar(qualified_percentages_by_category,
                          y=selected_category,
                          x='Percentage',
                          color=selected_job.replace('Total number of ', '').strip(),
                          color_discrete_map={
                              'Yes': '#4C78A8',
                              'None': 'goldenrod'},
                          text='Percentage',
                          barmode='stack',
                          orientation='h',
                          height=600,
                          labels={selected_category: 'Category', 'Count': 'Count'},
                          title=f'{selected_job.replace("Total number of ", "").strip().title()} per Category')

    category_fig.update_traces(texttemplate='%{text:.2f}%', textposition='inside')
    category_fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide',
                               xaxis_title='Percentage', yaxis_title='Category',
                               template='plotly_white')

    # Calculate overall percentages
    overall_percentages = calc_zero_perc(df, selected_job)

    # Plot the overall graph
    overall_fig = px.bar(overall_percentages,
                        x=overall_percentages.columns[0],
                        y='Count',
                        text='Percentage',
                        height=600,
                        color=overall_percentages.columns[0],
                        color_discrete_map={
                             f'Has {selected_job.replace("Total number of ", "").strip()}': '#4C78A8',
                             f'No {selected_job.replace("Total number of ", "").strip()}': 'goldenrod'},
                         title=f'Percentage of Archives with {selected_job.replace("Total number of ", "").strip().title()}')

    overall_fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
    overall_fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide',
                              xaxis_title= f'{selected_job.replace("Total number of ", "").strip()}', yaxis_title='Count',
                              template='plotly_white')

    # Plot binned data graph
    bin_fig = px.bar(archivistBins, x='archivists grouped', y='count',
             labels={'archivists grouped': 'archivists grouped', 'count': 'count'},
             title='Number of Qualified Archivists Grouped')
    bin_fig.update_traces(marker_color='#4C78A8')
    bin_fig.update_layout(xaxis_title='Number of Qualified Archivists (FTE)', yaxis_title='Count',
                  template='plotly_white')

    # Plot volunteer graph
    vol_df= volunteers_bar_chart(df, selected_category, vol_cols)

    vol_fig = px.bar(vol_df,
                     y=selected_category,
                     x='Count',
                     color='Volunteer Type',
                     barmode='stack',
                     orientation='h',
                     height=600,
                     color_discrete_map={
                              'Total number of onsite volunteers': '#4C78A8',
                              'Total number of online volunteers': 'rgb(136, 204, 238)'},
                     title='Onsite and Online Volunteers by {}'.format(selected_category))
    vol_fig.update_layout(yaxis={'categoryorder':'total ascending'}, margin=dict(l=30, r=30, t=100, b=20), template='plotly_white')


    return overall_fig, category_fig, bin_fig, vol_fig

if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>

In [ ]:
'''
with tabs
'''

#functions:
# Function to calculate percentages by category
def staff_percentages_by_category(df, cat_col, column_name):
    results = []

    # Filter out NaN values
    df_filtered = df[df[column_name].notna()]

    # Group by category
    grouped_data = df_filtered.groupby(cat_col)

    # Iterate over each group
    for category, group in grouped_data:
        total_archives = len(group)

        # Calculate counts for zero and non-zero qualified archivists
        zero_qualified_count = (group[column_name] == 0).sum()
        non_zero_qualified_count = (group[column_name] > 0).sum()

        # Calculate percentages
        zero_qualified_perc = round((zero_qualified_count / total_archives) * 100, 2)
        non_zero_qualified_perc = round((non_zero_qualified_count / total_archives) * 100, 2)

        job_name = column_name.replace('Total number of ', '').strip()

        # Append results to the list
        results.append({
            f'{cat_col}': category,
            job_name: 'None',
            'Count': zero_qualified_count,
            'Percentage': zero_qualified_perc
        })
        results.append({
            f'{cat_col}': category,
            job_name: 'Yes',
            'Count': non_zero_qualified_count,
            'Percentage': non_zero_qualified_perc
        })

    # Create a DataFrame from the results
    result_df = pd.DataFrame(results)
    return result_df

# Function to calculate % archives having zero staff vs some staff
def calc_zero_perc(df, column_name):
    # Filter out NaN values
    filtered_data = df[df[column_name].notna()]

    # Calculate total number of archives after filtering out NaN values
    total_archives = len(filtered_data)

    # Calculate counts for zero and non-zero values
    zero_count = (filtered_data[column_name] == 0).sum()
    non_zero_count = (filtered_data[column_name] > 0).sum()

    # Calculate percentages
    zero_percentage = round((zero_count / total_archives) * 100, 2)
    non_zero_percentage = round((non_zero_count / total_archives) * 100, 2)

    job_name = column_name.replace('Total number of ', '').strip()

    # Create a new DataFrame
    data = {
        job_name: [f'No {job_name}', f'Has {job_name}'],
        'Count': [zero_count, non_zero_count],
        'Percentage': [zero_percentage, non_zero_percentage]
    }
    result_df = pd.DataFrame(data)
    result_df = result_df.sort_values(by='Percentage', ascending=False)
    return result_df

def volunteers_bar_chart(df, group_by_column, target_cols):
    # Select only columns for summing, and include the grouping column

    grouped_df = df.groupby(group_by_column)[target_cols].sum().reset_index()

    # Melt the DataFrame to long format
    melted_df = pd.melt(grouped_df, id_vars=[group_by_column], value_vars=['Total number of onsite volunteers', 'Total number of online volunteers'],
                        var_name='Volunteer Type', value_name='Count')

    # Sort by the count of onsite volunteers
    melted_df['sort_order'] = melted_df.apply(lambda x: x['Count'] if x['Volunteer Type'] == 'Total number of onsite volunteers' else 0, axis=1)
    melted_df = melted_df.sort_values(by=['sort_order', group_by_column], ascending=[False, True])

    return melted_df

def calc_fzero_perc(df, column_name):
    # Filter out NaN values
    filtered_data = df[df[column_name].notna()]

    # Calculate total number of archives after filtering out NaN values
    total_archives = len(filtered_data)

    # Calculate counts for zero and non-zero values
    zero_count = (filtered_data[column_name] == 0).sum()
    non_zero_count = (filtered_data[column_name] > 0).sum()

    # Calculate percentages
    zero_percentage = round((zero_count / total_archives) * 100, 2)
    non_zero_percentage = round((non_zero_count / total_archives) * 100, 2)

    column_name = column_name.lower()

    # Create a new DataFrame
    data = {
        column_name: [f'Zero {column_name}', f'Some {column_name}'],
        'Count': [zero_count, non_zero_count],
        'Percentage': [zero_percentage, non_zero_percentage]
    }
    result_df = pd.DataFrame(data)
    return result_df

df=allData

# Initialize the Dash app
app = Dash(__name__)

# Layout with Tabs
app.layout = html.Div([
    dcc.Tabs(id='tabs-example', value='tab-1', children=[
        dcc.Tab(label='Staff Overview', value='tab-1'),
        dcc.Tab(label='Finance Overview', value='tab-2'),
    ]),
    html.Div(id='tabs-content-example')
])

@app.callback(Output('tabs-content-example', 'children'),
              Input('tabs-example', 'value'))
def render_content(tab):
    if tab == 'tab-1':
        # Staff Overview Tab content
        bins = [0, 1, 5, 10, 15, 20, 25, 30]
        labels = ['0-1', '1.1-5', '5.1-10', '10.1-15', '15.1-20', '20.1-25', '25.1-30']
        df['archivists grouped'] = pd.cut(df['Total number of qualified archivists'], bins=bins, labels=labels, include_lowest=True)
        archivistBins = df['archivists grouped'].value_counts(sort=False).reset_index().rename(columns={"index": "archivists", 0: "count"})

        # Data aggregation
        agg_cols = ['Total number of staff', 'Total number of qualified archivists', 'Total number of non-qualified archivists',
                    'Total number of qualified curators', 'Total number of records management staff', 'Total number of conservation staff',
                    'Total number of education and learning officers', 'Total number of project staff', 'Total number of onsite volunteers',
                    'Total number of online volunteers', 'Total number of volunteer hours']

        stats = df[agg_cols].agg(['count', 'max', 'min', 'sum','mean', 'median']).round(2)
        stats = stats.transpose()
        new_col_names = ['responses', 'highest', 'lowest', 'sum', 'average', 'median']
        stats.columns = new_col_names
        stats = stats.reset_index().rename(columns={'index': 'workforce'})

        df_table = stats
        vol_cols = ['Total number of onsite volunteers', 'Total number of online volunteers']



        return html.Div([
            html.Div(children='Staff Overview', style={'textAlign': 'center', 'color': 'blue', 'fontSize': 30, 'margin': '20px 0'}),

            html.Div(children=[
                dash_table.DataTable(data=df_table.to_dict('records'), page_size=5, style_table={'overflowX': 'auto', 'width': '70%', 'margin': '0 auto'},
                style_cell={'textAlign': 'center'})
            ]),

            html.Div([
              html.Div([
                    dcc.Graph(id='bin-graph', style={'height': '600px', 'margin': '10px auto'})
                ], style={'width': '80%'}
            )], style={'textAlign': 'centre'}),

            html.Div(children=[
                dcc.RadioItems(options=['Category', 'Postcode Region'],
                               value='Category', inline=True,
                               id='category-selector', style={'margin': '20px 0'})
            ], style={'textAlign': 'left'}),

            html.Div([
                dcc.Dropdown(
                    id='job-selector',
                    options=[{'label': col, 'value': col} for col in df.columns if col.startswith('Total number of ')],
                    value=[col for col in df.columns if col.startswith('Total number of ')][1],
                    style={'margin': '20px 0'}
                )
            ], style={'textAlign': 'left'}),

            html.Div([
                html.Div([
                    dcc.Graph(id='category-graph', style={'height': '600px', 'margin': '30px auto'})
                ], style={'width': '60%', 'display': 'inline-block', 'verticalAlign': 'top'}),

                html.Div([
                    dcc.Graph(id='overall-graph', style={'height': '600px', 'margin': '30px auto'})
                ], style={'width': '35%', 'display': 'inline-block', 'verticalAlign': 'top'})
            ], style={'textAlign': 'right'}),

            html.Div([
                dcc.Graph(id='volunteer-graph', style={'height': '500px'})
            ], style={'textAlign': 'center', 'padding': 10, 'flex':1})
        ])

    elif tab == 'tab-2':
        # Finance Overview Tab content
        mask = (df['TNA Region'] == 'Yorkshire')
        df.loc[mask,'Postcode Region'] = df.loc[mask,'TNA Region']
        df.loc[df["Postcode Region"] == "Yorkshire", "Postcode Region"] = "Yorkshire and The Humber"
        df.loc[df["Postcode Region"] == "Greater London", "Postcode Region"] = "London"

        agg_cols = [ 'Total revenue expenditure in the last financial year', 'Total capital expenditure in the last financial year', 'Total income in the last financial year']

        stats = df[agg_cols].agg(['count', 'max', 'min', 'sum','mean', 'median']).round(2)
        stats = stats.transpose()
        new_col_names = ['responses', 'highest', 'lowest', 'sum', 'average', 'median']
        stats.columns = new_col_names
        stats = stats.reset_index().rename(columns={'index': 'finance'})

        df_table = stats

        bins = [0, 1000, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 500000, 1000000, 2000000]
        labels = ['0-1000', '1001-10000', '10001-20000', '20001-30000', '30001-40000', '40001-50000', '50001-60000', '60001-70000', '70001-80000', '80001-90000', '90001-100000', '100001-500000', '500001-100000', '100001-200000']
        df['income grouped'] = pd.cut(df['Total income in the last financial year'], bins=bins, labels=labels, include_lowest=True)
        financeBins = df['income grouped'].value_counts(sort=False).reset_index().rename(columns={"index": "income", 0: "count"})


        return html.Div([
            html.Div(children='Finance Overview', style={'textAlign': 'center', 'color': 'green', 'fontSize': 30, 'margin': '20px 0'}),

            html.Div(children=[
                dash_table.DataTable(data=df_table.to_dict('records'), page_size=5, style_table={'overflowX': 'auto', 'width': '70%', 'margin': '0 auto'},
                style_cell={'textAlign': 'center'})
            ]),

            html.Div([
              html.Div([
                    dcc.Graph(id='fin-graph', style={'height': '600px', 'margin': '10px auto'})
                ], style={'width': '80%'}
            )], style={'textAlign': 'centre'}),

            html.Div([
                dcc.Dropdown(
                    id='f-selector',
                    options=[{'label': col, 'value': col} for col in df.columns if col.endswith('financial year')],
                    value=[col for col in df.columns if col.endswith('financial year')][2],
                    style={'margin': '20px 0'}
                )
            ], style={'textAlign': 'left'}),

              html.Div([
                html.Div([
                dcc.Graph(id='map-graph', style={'height': '600px'})
            ], style={'textAlign': 'center', 'padding': 10, 'flex':1}),

                html.Div([
                dcc.Graph(id='fin-category-graph', style={'height': '600px', 'margin': '10px auto'})
                ], style={'width': '60%', 'display': 'inline-block', 'verticalAlign': 'top'}),

                html.Div([
                dcc.Graph(id='zero-fin-graph', style={'height': '600px', 'margin': '10px auto'})
                ], style={'width': '35%', 'display': 'inline-block', 'verticalAlign': 'top'})
            ], style={'textAlign': 'right'})
        ])

@app.callback(
    [Output('category-graph', 'figure'),
     Output('overall-graph', 'figure'),
     Output('bin-graph', 'figure'),
     Output('volunteer-graph', 'figure')],
    [Input('category-selector', 'value'),
     Input('job-selector', 'value')]
)
def update_staff_graphs(selected_category, selected_job):
     # Calculate percentages by category using the selected job
    qualified_percentages_by_category = staff_percentages_by_category(df, selected_category, selected_job)
    qualified_percentages_by_category = qualified_percentages_by_category.sort_values(by='Percentage', ascending=False)

    # Plot the category graph
    category_fig = px.bar(qualified_percentages_by_category,
                          y=selected_category,
                          x='Percentage',
                          color=selected_job.replace('Total number of ', '').strip(),
                          color_discrete_map={
                              'Yes': '#4C78A8',
                              'None': 'goldenrod'},
                          text='Percentage',
                          barmode='stack',
                          orientation='h',
                          height=600,
                          labels={selected_category: 'Category', 'Count': 'Count'},
                          title=f'{selected_job.replace("Total number of ", "").strip().title()} per Category')

    category_fig.update_traces(texttemplate='%{text:.2f}%', textposition='inside')
    category_fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide',
                               xaxis_title='Percentage', yaxis_title='Category',
                               template='plotly_white')

    # Calculate overall percentages
    overall_percentages = calc_fzero_perc(df, selected_job)

    # Plot the overall graph
    overall_fig = px.bar(overall_percentages,
                        x=overall_percentages.columns[0],
                        y='Count',
                        text='Percentage',
                        height=600,
                        color=overall_percentages.columns[0],
                        color_discrete_map={
                             f'Has {selected_job.replace("Total number of ", "").strip()}': '#4C78A8',
                             f'No {selected_job.replace("Total number of ", "").strip()}': 'goldenrod'},
                         title=f'Percentage of Archives with {selected_job.replace("Total number of ", "").strip().title()}')

    overall_fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
    overall_fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide',
                              xaxis_title= f'{selected_job.replace("Total number of ", "").strip()}', yaxis_title='Count',
                              template='plotly_white')

    # Plot binned data graph
    bin_fig = px.bar(archivistBins, x='archivists grouped', y='count',
             labels={'archivists grouped': 'archivists grouped', 'count': 'count'},
             title='Number of Qualified Archivists Grouped')
    bin_fig.update_traces(marker_color='#4C78A8')
    bin_fig.update_layout(xaxis_title='Number of Qualified Archivists (FTE)', yaxis_title='Count',
                  template='plotly_white')

    # Plot volunteer graph
    vol_df= volunteers_bar_chart(df, selected_category, vol_cols)

    vol_fig = px.bar(vol_df,
                     y=selected_category,
                     x='Count',
                     color='Volunteer Type',
                     barmode='stack',
                     orientation='h',
                     height=600,
                     color_discrete_map={
                              'Total number of onsite volunteers': '#4C78A8',
                              'Total number of online volunteers': 'rgb(136, 204, 238)'},
                     title='Onsite and Online Volunteers by {}'.format(selected_category))
    vol_fig.update_layout(yaxis={'categoryorder':'total ascending'}, margin=dict(l=30, r=30, t=100, b=20), template='plotly_white')


    return category_fig, overall_fig, bin_fig, vol_fig

@app.callback(
    [Output('map-graph', 'figure'),
     Output('fin-graph', 'figure'),
     Output('fin-category-graph', 'figure'),
     Output('zero-fin-graph', 'figure')],
    Input('f-selector', 'value')
)
def update_finance_graphs(selected_job):
    df1 = df.groupby('Postcode Region')[selected_job].sum().round(2).reset_index()
    merged_gdf = gdf_sp.merge(df1, left_on='RGN23NM', right_on='Postcode Region')
    merged_gdf = merged_gdf.to_crs("WGS84")
    merged_gdf.set_index('RGN23NM', inplace=True)

    map_fig = px.choropleth(
    merged_gdf,
    geojson=merged_gdf.geometry,
    locations=merged_gdf.index,
    #featureidkey="properties.RGN23CD",  # Adjust this to match the GeoJSON properties key
    color=selected_job,
    color_continuous_scale="gnbu",
    #range_color=(merged_gdf['Total income in the last financial year'].min(), merged_gdf['Total income in the last financial year'].max()),
    labels={selected_job: selected_job, 'RGN23NM': 'Region'},
    title=f'{selected_job} by Region'
)

  # Update the layout to display the map
    map_fig.update_geos(fitbounds='geojson', visible=False)
    map_fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})

    fin_fig = px.bar(financeBins, x='income grouped', y='count',
             labels={'income grouped': '	Income grouped', 'count': 'Count'},
             title='Total Income Grouped')
    fin_fig.update_layout(xaxis_title='	Income grouped', yaxis_title='Count',
                  template='plotly_white',
                  width=800, height=600)
    fin_fig.update_traces(marker_color='rgb(102,166,30)')


    # Calculate overall percentages
    zero_percentages = calc_fzero_perc(df, selected_job)

    # Plot the overall graph
    zero_fig = px.bar(zero_percentages,
                        x=zero_percentages.columns[0],
                        y='Count',
                        text='Percentage',
                        height=600,
                        color=zero_percentages.columns[0],
                        color_discrete_map={
                             f'Has {selected_job.replace("in the last financial year", "").strip()}': 'rgb(153, 201, 69)',
                             f'No {selected_job.replace("in the last financial year", "").strip()}': 'rgb(229, 134, 6)'},
                         title=f'Percentage of Archives with<br>{selected_job.title()}')

    zero_fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
    zero_fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide',
                              xaxis_title='', yaxis_title='Count',
                              template='plotly_white')
    zero_fig.update_xaxes(showticklabels=False)

    df1 = df.groupby('Category')[selected_job].sum().round(2).reset_index()
    df1 = df1.sort_values(by=selected_job, ascending=True)
    fcat_fig = px.bar(df1, x=selected_job, y='Category',text=selected_job,
             labels={selected_job: f'{selected_job.replace("in the last financial year", "").strip()}'},
             title=f'{selected_job.replace("in the last financial year", "").strip().title()} Per Category')
    fcat_fig.update_traces(texttemplate='%{text:.3s}', textposition='none', marker_color='rgb(166,216,84)')
    fcat_fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide',
                  #width=1100,
                  xaxis_title=f'{selected_job.replace("in the last financial year", "").strip()}', yaxis_title='Category',
                  template='plotly_white')


    return map_fig, fin_fig, fcat_fig, zero_fig

if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>

In [ ]:
print(px.colors.qualitative.Bold)

['rgb(127, 60, 141)', 'rgb(17, 165, 121)', 'rgb(57, 105, 172)', 'rgb(242, 183, 1)', 'rgb(231, 63, 116)', 'rgb(128, 186, 90)', 'rgb(230, 131, 16)', 'rgb(0, 134, 149)', 'rgb(207, 28, 144)', 'rgb(249, 123, 114)', 'rgb(165, 170, 153)']
